In [1]:
!pip install sentence-transformers


In [2]:
import random
import math
import nltk
import re
import math
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
random.seed(0)  # Set a fixed seed for reproducibility


In [58]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

sentences = ["I am sad"]

model_outputs = classifier(sentences)
print(model_outputs[0])
print(mental_health_score(model_outputs[0]))

[{'label': 'sadness', 'score': 0.9104166626930237}, {'label': 'disappointment', 'score': 0.06846530735492706}, {'label': 'grief', 'score': 0.03181026130914688}, {'label': 'neutral', 'score': 0.026156943291425705}, {'label': 'remorse', 'score': 0.02074185200035572}, {'label': 'annoyance', 'score': 0.01124617736786604}, {'label': 'realization', 'score': 0.01070040836930275}, {'label': 'caring', 'score': 0.00934799574315548}, {'label': 'approval', 'score': 0.008505051955580711}, {'label': 'love', 'score': 0.007716427091509104}, {'label': 'admiration', 'score': 0.007372341118752956}, {'label': 'disapproval', 'score': 0.00708762789145112}, {'label': 'anger', 'score': 0.006395332049578428}, {'label': 'joy', 'score': 0.005612883251160383}, {'label': 'amusement', 'score': 0.0055735232308506966}, {'label': 'nervousness', 'score': 0.004651421681046486}, {'label': 'disgust', 'score': 0.004631553310900927}, {'label': 'curiosity', 'score': 0.004111076705157757}, {'label': 'desire', 'score': 0.00387

In [57]:
def mental_health_score(emotions, threshold=0.5):
    # Define a set of labels associated with mental health concerns
    mental_health_labels = {
        'sadness', 'disappointment', 'annoyance', 'disapproval',
        'nervousness', 'remorse', 'embarrassment', 'anger',
        'disgust', 'grief', 'confusion', 'fear'
    }

    # Initialize variables to calculate the score
    total_score = 0
    max_score = 0

    # Iterate over the emotions
    for emotion in emotions:
        label = emotion['label']
        score = emotion['score']
        if label in mental_health_labels:
            total_score += score
        max_score += score

    # Normalize the total score by the maximum possible score
    normalized_score = total_score / max_score if max_score > 0 else 0

    return normalized_score


In [3]:

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def lemmatize(input):
  lemmatizer = WordNetLemmatizer()

  sentence = " ".join(input)

  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))

  #>[('the', 'DT'), ('cat', 'NN'), ('is', 'VBZ'), ('sitting', 'VBG'), ('with', 'IN'),
  # ('the', 'DT'), ('bats', 'NNS'), ('on', 'IN'), ('the', 'DT'), ('striped', 'JJ'),
  # ('mat', 'NN'), ('under', 'IN'), ('many', 'JJ'), ('flying', 'VBG'), ('geese', 'JJ')]

  # As you may have noticed, the above pos tags are a little confusing.

  # we use our own pos_tagger function to make things simpler to understand.
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  #>[('the', None), ('cat', 'n'), ('is', 'v'), ('sitting', 'v'), ('with', None),
  # ('the', None), ('bats', 'n'), ('on', None), ('the', None), ('striped', 'a'),
  # ('mat', 'n'), ('under', None), ('many', 'a'), ('flying', 'v'), ('geese', 'a')]

  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
      if tag is None:
          # if there is no available tag, append the token as is
          lemmatized_sentence.append(word)
      else:
          # else use the tag to lemmatize the token
          lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  return lemmatized_sentence


In [45]:
class Vertex:
    def __init__(self, name, arr_words):
        self.name = name
        self.edge = {}  # {name: edge weight}
        self.weight = 1  # 1-10
        self.words = {}  # {word: count}
        for word in arr_words:
            if word in self.words:
                self.words[word] += 1
            else:
                self.words[word] = 1
        self.num_words = sum(self.words.values())

    def connection(self, connect_name, weight):
        self.edge[connect_name] = weight


In [87]:
class SummaryBot:
    def __init__(self):
        self.vertices = []
        self.dampening_factor = 0.85
        self.num_vertices = 0
        self.original_sentences = []
        self.filtered_sentences = []
        self.error_threshold = 0.01
        self.unconnected_vertices = set()
        self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


    def run(self, text, num_return_sentences=None, test_statistics=False):
        if not isinstance(text, str):
            raise TypeError('Ensure that you pass valid values into SummaryBot.run')

        self._process_string(text)
        self._initialize()
        last_v0 = None
        cur_v0 = None
        error_level = 10
        while error_level > self.error_threshold:
            self._update_all_vertex_weights()
            if cur_v0 is None:
                cur_v0 = self.vertices[0].weight
            else:
                last_v0 = cur_v0
                cur_v0 = self.vertices[0].weight
                error_level = abs(cur_v0 - last_v0)

        if not num_return_sentences:
            num_return_sentences = self._find_best_num_sentences()

        output = self.get_top_sentences(num_return_sentences)



        if test_statistics:
            output += self.summary()
        return output

    def _find_best_num_sentences(self):
        if len(self.vertices) <= 3:
            return len(self.vertices)
        return round(1.3 * math.log(len(self.original_sentences)))

    def get_top_sentences(self, num_sentences):
        self._sort_vertices_by_weight()

        top_sentences_arr = []
        if num_sentences > self.num_vertices:
            return '. '.join(self.original_sentences)
        else:
            for sentence_out in range(num_sentences):
                top_sentences_arr.append(self.vertices[sentence_out])

            top_sentences_arr.sort(key=lambda vert: vert.name)
            out = [self.original_sentences[vert.name] for vert in top_sentences_arr]

        if any(vert.name == len(self.original_sentences) - 1 for vert in top_sentences_arr):
            return '. '.join(out)
        return '. '.join(out) + '.'

    def _process_string(self, text):
        self.original_sentences = re.split(r'[.?!]', text)
        self.filtered_sentences = [sentence.replace(r'[^a-zA-Z0-9 ]', ' ') for sentence in self.original_sentences]
        self.filtered_sentences = [sentence for sentence in self.filtered_sentences if sentence]

        for sentence in self.filtered_sentences:
            word_list = [word.lower() for word in sentence.split() if word and len(word) != 1 and word not in stop_words]
            # print(word_list)
            word_list=lemmatize(word_list)
            # print(word_list)

            self._add_vertex(word_list)

    def _add_vertex(self, word_arr):
        self.vertices.append(Vertex(self.num_vertices, word_arr))
        self.num_vertices += 1

    def _initialize(self):
        for vertex_num in range(len(self.vertices) - 1):
            for inner_vertex in range(vertex_num + 1, len(self.vertices)):
                similarity = self._find_similarity(self.vertices[vertex_num], self.vertices[inner_vertex])
                self.vertices[vertex_num].connection(inner_vertex, similarity)
                self.vertices[inner_vertex].connection(vertex_num, similarity)

    def _update_all_vertex_weights(self):
        total_edge_weights = self._get_edge_totals()
        for vertex_num in range(self.num_vertices):
            temp_sum = 0
            for other_vertex in range(self.num_vertices):
                if other_vertex != vertex_num and other_vertex not in self.unconnected_vertices:
                    temp_sum += (self.vertices[vertex_num].edge[other_vertex] / total_edge_weights[other_vertex] * self.vertices[other_vertex].weight)
            self.vertices[vertex_num].weight = ((1 - self.dampening_factor) + self.dampening_factor * temp_sum)+0.2*mental_health_score(classifier(self.original_sentences[self.vertices[vertex_num].name])[0])

    def _get_edge_totals(self):
        out = []
        for vertex_num in range(self.num_vertices):
            temp_sum = sum(self.vertices[vertex_num].edge.values())
            if not temp_sum:
                self.vertices[vertex_num].weight = 0
                self.unconnected_vertices.add(vertex_num)
            out.append(temp_sum)
        return out

    # def _find_similarity(self, vert1, vert2):
    #     overlap = 0
    #     for word in vert1.words:
    #         if word in vert2.words:
    #             overlap += math.pow(vert1.words[word] * vert2.words[word], 0.7)

    #     return overlap / (math.log(vert1.num_words) + math.log(vert2.num_words)+1)

    # def _find_similarity(self, vert1, vert2):
    #     """
    #     Computes the cosine similarity between two vertices based on their word frequencies.

    #     Args:
    #         vert1 (Vertex): The first vertex.
    #         vert2 (Vertex): The second vertex.

    #     Returns:
    #         float: The cosine similarity between the two vertices.
    #     """
    #     # Convert the word dictionaries to vectors
    #     words1 = vert1.words
    #     words2 = vert2.words

    #     # Create a set of all words in both vertices
    #     all_words = list(set(words1.keys()).union(set(words2.keys())))

    #     # Create vectors for each vertex
    #     vector1 = [words1.get(word, 0) for word in all_words]
    #     vector2 = [words2.get(word, 0) for word in all_words]
    #     # Compute the cosine similarity
    #     vec1 = np.array(vector1).reshape(1, -1)
    #     vec2 = np.array(vector2).reshape(1, -1)
    #     similarity = cosine_similarity(vec1, vec2)[0][0]

    #     return similarity

    # def _find_similarity(self, vert1, vert2):
    #   # Prepare the data for TF-IDF
    #   sentences = [self.original_sentences[vert1.name], self.original_sentences[vert2.name]]
    #   vectorizer = TfidfVectorizer()
    #   tfidf_matrix = vectorizer.fit_transform(sentences)

    #   # Calculate cosine similarity
    #   similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    #   return similarity

    def _find_similarity(self, vert1, vert2):
        # Get sentence embeddings using a pre-trained SentenceTransformer model
        sentence1 = self.original_sentences[vert1.name]
        sentence2 = self.original_sentences[vert2.name]
        embedding1 = self.model.encode(sentence1, convert_to_tensor=True)
        embedding2 = self.model.encode(sentence2, convert_to_tensor=True)

        # Compute cosine similarity
        similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
        return similarity

    # def _find_similarity(self, vert1, vert2):
    #     # Combine TF-IDF cosine similarity and Sentence Embedding cosine similarity
    #     tfidf_similarity = self._word_frequency_cosine_similarity(vert1, vert2)
    #     embedding_similarity = self._sentence_embedding_cosine_similarity(vert1, vert2)
    #     # Weighted average or any other combination method can be applied
    #     combined_similarity = 0.0*tfidf_similarity + 1.0*embedding_similarity
    #     return combined_similarity
    def _sort_vertices_by_weight(self):
        self.vertices.sort(key=lambda vert: vert.weight, reverse=True)


In [ ]:
bot = SummaryBot()
summary = bot.run("The solar system is a fascinating and complex entity, consisting of the Sun and all the objects that orbit around it, including the eight planets and their moons, as well as dwarf planets, comets, and asteroids. The Sun, a star at the center, provides the necessary heat and light to sustain life on Earth. Each planet has unique characteristics, such as Mars with its red surface, Jupiter with its Great Red Spot, and Saturn with its rings. Understanding the solar system helps scientists learn more about the origins of the universe and the potential for life beyond Earth.")
print(summary)

The solar system is a fascinating and complex entity, consisting of the Sun and all the objects that orbit around it, including the eight planets and their moons, as well as dwarf planets, comets, and asteroids. The Sun, a star at the center, provides the necessary heat and light to sustain life on Earth. Each planet has unique characteristics, such as Mars with its red surface, Jupiter with its Great Red Spot, and Saturn with its rings. Understanding the solar system helps scientists learn more about the origins of the universe and the potential for life beyond Earth.


In [ ]:
The solar system is a fascinating and complex entity, consisting of the Sun and all the objects that orbit around it, including the eight planets and their moons, as well as dwarf planets, comets, and asteroids. The Sun, a star at the center, provides the necessary heat and light to sustain life on Earth. Each planet has unique characteristics, such as Mars with its red surface, Jupiter with its Great Red Spot, and Saturn with its rings. Understanding the solar system helps scientists learn more about the origins of the universe and the potential for life beyond Earth.
The solar system is a fascinating and complex entity, consisting of the Sun and all the objects that orbit around it, including the eight planets and their moons, as well as dwarf planets, comets, and asteroids. The Sun, a star at the center, provides the necessary heat and light to sustain life on Earth. Each planet has unique characteristics, such as Mars with its red surface, Jupiter with its Great Red Spot, and Saturn with its rings. Understanding the solar system helps scientists learn more about the origins of the universe and the potential for life beyond Earth.


## consine

In [75]:
bot = SummaryBot()
summary = bot.run("Hello, I am depressed. yes, but no one loves me I should kill my self. How to find the right person. may be I will. I hate everyone I hate every body, no one cares about me. I just think I should die. yes, but I think family or friends wont help me. may be I will try to talk with my family. I will try.")
print(summary)

 yes, but no one loves me I should kill my self.  yes, but I think family or friends wont help me.  may be I will try to talk with my family.


In [78]:
bot = SummaryBot()
summary = bot.run("Hello, I am depressed. yes, but no one loves me I should kill my self. How to find the right person. may be I will. I hate everyone I hate every body, no one cares about me. I just think I should die. yes, but I think family or friends wont help me. may be I will try to talk with my family. I will try.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 yes, but no one loves me I should kill my self.  yes, but I think family or friends wont help me.  may be I will try to talk with my family.


In [72]:
bot = SummaryBot()
summary = bot.run("Hello, I am depressed. yes, but no one loves me I should kill my self. How to find the right person. may be I will. I hate everyone I hate every body, no one cares about me. I just think I should die. yes, but I think family or friends wont help me. may be I will try to talk with my family. I will try.")
print(summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 yes, but no one loves me I should kill my self.  I just think I should die.  may be I will try to talk with my family.


In [54]:
bot = SummaryBot()
summary = bot.run("Hello, I am depressed. yes, but no one loves me I should kill my self. How to find the right person. may be I will. I hate everyone I hate every body, no one cares about me. I just think I should die. yes, but I think family or friends wont help me. may be I will try to talk with my family. I will try.")
print(summary)


 yes, but no one loves me I should kill my self.  may be I will.  may be I will try to talk with my family.


In [76]:
bot = SummaryBot()
summary = bot.run("I guess. Just been feeling a bit overwhelmed lately. Well, work has been really stressful, and I've been having trouble sleeping. Not really. I've tried a few things, like deep breathing and taking walks, but nothing seems to help much. No do you think seeing a therapist will be help? Yeah, I think that could be really helpful. I'm willing to give it a try. Thanks, doc. I appreciate it. It feels good to know I have someone to talk to about this stuff. Do you like video games? you may should try.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 Well, work has been really stressful, and I've been having trouble sleeping.  I've tried a few things, like deep breathing and taking walks, but nothing seems to help much.  Yeah, I think that could be really helpful.


In [65]:
bot = SummaryBot()
summary = bot.run("I guess. Just been feeling a bit overwhelmed lately. Well, work has been really stressful, and I've been having trouble sleeping. Not really. I've tried a few things, like deep breathing and taking walks, but nothing seems to help much. No do you think seeing a therapist will be help? Yeah, I think that could be really helpful. I'm willing to give it a try. Thanks, doc. I appreciate it. It feels good to know I have someone to talk to about this stuff. Do you like video games? you may should try.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 Well, work has been really stressful, and I've been having trouble sleeping.  I've tried a few things, like deep breathing and taking walks, but nothing seems to help much.  Yeah, I think that could be really helpful.


In [53]:
bot = SummaryBot()
summary = bot.run("I guess. Just been feeling a bit overwhelmed lately. Well, work has been really stressful, and I've been having trouble sleeping. Not really. I've tried a few things, like deep breathing and taking walks, but nothing seems to help much. No do you think seeing a therapist will be help? Yeah, I think that could be really helpful. I'm willing to give it a try. Thanks, doc. I appreciate it. It feels good to know I have someone to talk to about this stuff. Do you like video games? you may should try.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 Well, work has been really stressful, and I've been having trouble sleeping.  No do you think seeing a therapist will be help.  I'm willing to give it a try.


In [73]:
bot = SummaryBot()
summary = bot.run("I guess. Just been feeling a bit overwhelmed lately. Well, work has been really stressful, and I've been having trouble sleeping. Not really. I've tried a few things, like deep breathing and taking walks, but nothing seems to help much. No do you think seeing a therapist will be help? Yeah, I think that could be really helpful. I'm willing to give it a try. Thanks, doc. I appreciate it. It feels good to know I have someone to talk to about this stuff. Do you like video games? you may should try.")
print(summary)

 Yeah, I think that could be really helpful.  I'm willing to give it a try.  It feels good to know I have someone to talk to about this stuff.


In [80]:
bot = SummaryBot()
summary = bot.run("Not great, to be honest. User've been feeling very low and hopeless lately. Yes, but User don't see any point in living anymore. User just feel no one loves User. what should User do? User don't believe Assistant. Nothing ever changes for User. User've tried everything, and nothing works. That is maybe good. Thank Assistant. User don't know. User guess User can try. User have nothing to lose. Haha right. Mmmm User like to go clubbing and swim. Yes, swimming changes User's mood. Thank Assistant, goodbye.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 User just feel no one loves User.  what should User do.  User don't believe Assistant.  User guess User can try.


In [82]:
bot = SummaryBot()
summary = bot.run("Not great, to be honest. User've been feeling very low and hopeless lately. Yes, but User don't see any point in living anymore. User just feel no one loves User. what should User do? User don't believe Assistant. Nothing ever changes for User. User've tried everything, and nothing works. That is maybe good. Thank Assistant. User don't know. User guess User can try. User have nothing to lose. Haha right. Mmmm User like to go clubbing and swim. Yes, swimming changes User's mood. Thank Assistant, goodbye.")
print(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


 User just feel no one loves User.  what should User do.  User don't believe Assistant.  User guess User can try.


In [88]:
bot = SummaryBot()
summary = bot.run("Not great, to be honest. User've been feeling very low and hopeless lately. Yes, but User don't see any point in living anymore. User just feel no one loves User. what should User do? User don't believe Assistant. Nothing ever changes for User. User've tried everything, and nothing works. That is maybe good. Thank Assistant. User don't know. User guess User can try. User have nothing to lose. Haha right. Mmmm User like to go clubbing and swim. Yes, swimming changes User's mood. Thank Assistant, goodbye.")
print(summary)

 Yes, but User don't see any point in living anymore.  User just feel no one loves User.  what should User do.  User have nothing to lose.


In [86]:
bot = SummaryBot()
summary = bot.run("Not great, to be honest. User've been feeling very low and hopeless lately. Yes, but User don't see any point in living anymore. User just feel no one loves User. what should User do? User don't believe Assistant. Nothing ever changes for User. User've tried everything, and nothing works. That is maybe good. Thank Assistant. User don't know. User guess User can try. User have nothing to lose. Haha right. Mmmm User like to go clubbing and swim. Yes, swimming changes User's mood. Thank Assistant, goodbye.")
print(summary)

 Yes, but User don't see any point in living anymore.  what should User do.  User don't know.  User have nothing to lose.


In [47]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
all_words= ["i","love","better"]
words_lemmatized = [lemmatizer.lemmatize(word.lower(), pos="a") for word in all_words]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
words_lemmatized

['i', 'love', 'good']

In [25]:

sentences = ["I do not like sadness"]

model_outputs = classifier(sentences)
print(model_outputs[0])
print(mental_health_score(model_outputs[0]))

[{'label': 'disapproval', 'score': 0.5441403388977051}, {'label': 'sadness', 'score': 0.3584756851196289}, {'label': 'disappointment', 'score': 0.13139913976192474}, {'label': 'annoyance', 'score': 0.04687810689210892}, {'label': 'neutral', 'score': 0.03726961463689804}, {'label': 'anger', 'score': 0.02318556047976017}, {'label': 'realization', 'score': 0.011338314972817898}, {'label': 'approval', 'score': 0.01024869829416275}, {'label': 'disgust', 'score': 0.009279333986341953}, {'label': 'caring', 'score': 0.008014443330466747}, {'label': 'remorse', 'score': 0.006530173122882843}, {'label': 'love', 'score': 0.006400663871318102}, {'label': 'joy', 'score': 0.004949627444148064}, {'label': 'confusion', 'score': 0.0046013290993869305}, {'label': 'grief', 'score': 0.004066526889801025}, {'label': 'embarrassment', 'score': 0.003994239494204521}, {'label': 'fear', 'score': 0.003558812430128455}, {'label': 'nervousness', 'score': 0.0032670439686626196}, {'label': 'desire', 'score': 0.002978